<a href="https://colab.research.google.com/github/DavidMercadoFaustino/Modelagem-de-Previsao-de-Inadimplencia/blob/main/Tratamento-Exploracao-dados/Explora%C3%A7%C3%A3o_e_Tratamento_dos_dados_Home_Credit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
! pip install kaggle

In [46]:
! mkdir ~/.kaggle
! cp /content/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [47]:
! kaggle competitions download -c home-credit-default-risk -p '/content/home-credit-default-risk'

home-credit-default-risk.zip: Skipping, found more recently modified local copy (use --force to force download)


In [48]:
import zipfile
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer # Importante para IterativeImputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder # Embora usados antes, boa prática ter importados

In [49]:
# Mostra para Python onde estão os arquivos do dataset
caminho_projeto = '/content/home-credit-default-risk'
if not os.path.exists(caminho_projeto):
    os.mkdir(caminho_projeto)
    print(f"Pasta criada em: {caminho_projeto}")
else:
    print(f"A pasta já existe em: {caminho_projeto}")

# Criar pasta para imagens se não existir
imagens_dir = 'imagens' # Nome da pasta para imagens
if not os.path.exists(imagens_dir):
    os.makedirs(imagens_dir)
    print(f"Pasta '{imagens_dir}' criada para salvar gráficos.")
else:
    print(f"Pasta '{imagens_dir}' já existe.")

caminho = os.getcwd()
# Todos arquivos NA PASTA INFORMADA
arquivos = os.listdir(caminho_projeto)
for arquivo in arquivos:
    if arquivo.endswith('.zip'):
        caminho_completo = os.path.join(caminho_projeto, arquivo)




A pasta já existe em: /content/home-credit-default-risk
Pasta 'imagens' já existe.


In [50]:
with zipfile.ZipFile(caminho_completo, 'r') as zip_ref:
    zip_ref.extractall(caminho_projeto)
print("Arquivos extraídos com sucesso.")

Arquivos extraídos com sucesso.


In [51]:
df_application_train = pd.read_csv(caminho_projeto + '/application_train.csv')
df_application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
df_description = pd.read_csv(caminho_projeto + '/HomeCredit_columns_description.csv', encoding='latin1')
df_description.head()

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN


In [53]:
tipos_credito = df_application_train['NAME_CONTRACT_TYPE'].unique()
print(f"Tipos de Crédito: {tipos_credito}")

Tipos de Crédito: ['Cash loans' 'Revolving loans']


In [54]:
# Configurar o estilo para os gráficos
sns.set_style("whitegrid")

# Criar o gráfico de contagem para a variável TARGET, separando por tipo de contrato
plt.figure(figsize=(10, 6)) # Aumentado o tamanho para melhor visualização com 'hue'
ax = sns.countplot(x='TARGET', hue='NAME_CONTRACT_TYPE', data=df_application_train, palette='viridis')
plt.title('Distribuição da Inadimplência por Tipo de Contrato')
plt.xlabel('Status do Empréstimo (0: Pago, 1: Inadimplente)')
plt.ylabel('Número de Clientes')
plt.legend(title='Tipo de Contrato') # Adiciona legenda para o 'hue'

# Adicionar os valores na parte superior das barras
for p in ax.patches:
    ax.annotate(f'{p.get_height():.0f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 5), textcoords='offset points', fontsize=9)

# --- Salvar o Gráfico ---
# Salvar o gráfico na pasta 'imagens'
plt.savefig(os.path.join(imagens_dir, 'target_distribution_by_contract_type.png'))
plt.close() # Fechar o plot para liberar memória

print("Gráfico 'target_distribution.png' gerado com sucesso!")

Gráfico 'target_distribution.png' gerado com sucesso!


In [55]:
# Configurar o estilo para os gráficos
sns.set_style("whitegrid")

# Criar o gráfico de contagem para a variável TARGET, separando por genero
plt.figure(figsize=(10, 6)) # Aumentado o tamanho para melhor visualização com 'hue'
ax = sns.countplot(x='TARGET', hue='CODE_GENDER', data=df_application_train, palette='viridis')
plt.title('Distribuição da Inadimplência por Genero')
plt.xlabel('Status do Empréstimo (0: Pago, 1: Inadimplente)')
plt.ylabel('Número de Clientes')
plt.legend(title='Genero do toomante de credito') # Adiciona legenda para o 'hue'

# Adicionar os valores na parte superior das barras
for p in ax.patches:
    ax.annotate(f'{p.get_height():.0f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 5), textcoords='offset points', fontsize=9)

# --- Salvar o Gráfico ---
# Salvar o gráfico na pasta 'imagens'
plt.savefig(os.path.join(imagens_dir, 'target_distribution_by_gender.png'))
plt.close() # Fechar o plot para liberar memória

print("Gráfico 'target_distribution_by_gender.png' gerado com sucesso!")

Gráfico 'target_distribution_by_gender.png' gerado com sucesso!


In [56]:
# Ver o tipo de dado de cada coluna
print("\nTipos de Dados por Coluna:")
print(df_application_train.dtypes.head(20))

# Ou, para uma visão mais resumida de tipos únicos
print("\nContagem de Tipos de Dados:")
print(df_application_train.dtypes.value_counts())


Tipos de Dados por Coluna:
SK_ID_CURR                      int64
TARGET                          int64
NAME_CONTRACT_TYPE             object
CODE_GENDER                    object
FLAG_OWN_CAR                   object
FLAG_OWN_REALTY                object
CNT_CHILDREN                    int64
AMT_INCOME_TOTAL              float64
AMT_CREDIT                    float64
AMT_ANNUITY                   float64
AMT_GOODS_PRICE               float64
NAME_TYPE_SUITE                object
NAME_INCOME_TYPE               object
NAME_EDUCATION_TYPE            object
NAME_FAMILY_STATUS             object
NAME_HOUSING_TYPE              object
REGION_POPULATION_RELATIVE    float64
DAYS_BIRTH                      int64
DAYS_EMPLOYED                   int64
DAYS_REGISTRATION             float64
dtype: object

Contagem de Tipos de Dados:
float64    65
int64      41
object     16
Name: count, dtype: int64


In [57]:
df_application_train.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,...,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,...,0.008130,0.000595,0.000507,0.000335,0.006402,0.007000,0.034362,0.267395,0.265474,1.899974
std,102790.175348,0.272419,0.722121,2.371231e+05,4.024908e+05,14493.737315,3.694465e+05,0.013831,4363.988632,141275.766519,...,0.089798,0.024387,0.022518,0.018299,0.083849,0.110757,0.204685,0.916002,0.794056,1.869295
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17912.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189145.500000,0.000000,0.000000,1.125000e+05,2.700000e+05,16524.000000,2.385000e+05,0.010006,-19682.000000,-2760.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278202.000000,0.000000,0.000000,1.471500e+05,5.135310e+05,24903.000000,4.500000e+05,0.018850,-15750.000000,-1213.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367142.500000,0.000000,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12413.000000,-289.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,456255.000000,1.000000,19.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7489.000000,365243.000000,...,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


**TARGET**: A coluna TARGET tem uma mean de aproximadamente 0.0807. Como o TARGET é 0 ou 1, isso significa que cerca de 8% dos clientes neste dataset entraram em default, e os outros 92% pagaram. Isso confirma que temos um problema de dados desbalanceados, um desafio comum em problemas de risco de crédito!

**AMT_INCOME_TOTAL**: A renda total dos clientes varia bastante, com uma média de 168.797 e um máximo de impressionantes 117.000.000 (provavelmente um outlier ou algum erro de entrada, mas vale a pena investigar).

**DAYS_BIRTH**: O valor médio é -16036 dias, o que, dividido por 365, é cerca de -43 anos. Isso é o esperado, já que os dias são contados do dia atual para trás.

**DAYS_EMPLOYED**: Aqui vemos um max de 365243.0, que é um número muito grande de dias (quase 1000 anos!). Isso é um valor anômalo comum neste dataset, que geralmente indica pessoas desempregadas ou com status de emprego "infinito". Precisaremos tratar isso.

In [58]:
# Contar quantos valores são iguais a 365243 em DAYS_EMPLOYED
anomalous_employment_count = df_application_train[df_application_train['DAYS_EMPLOYED'] == 365243]['DAYS_EMPLOYED'].count()
total_count = len(df_application_train)
percentage_anomalous = (anomalous_employment_count / total_count) * 100

print(f"Número de valores anômalos (365243) em 'DAYS_EMPLOYED': {anomalous_employment_count}")
print(f"Porcentagem de valores anômalos em 'DAYS_EMPLOYED': {percentage_anomalous:.2f}%")

Número de valores anômalos (365243) em 'DAYS_EMPLOYED': 55374
Porcentagem de valores anômalos em 'DAYS_EMPLOYED': 18.01%


In [59]:
df_employed_clean = df_application_train[df_application_train['DAYS_EMPLOYED'] != 365243].copy()

# Converter dias para anos para melhor interpretabilidade (e para valores positivos)
df_employed_clean['YEARS_EMPLOYED'] = df_employed_clean['DAYS_EMPLOYED'] / -365

# 1. Visualizar a distribuição de 'YEARS_EMPLOYED' (sem os anômalos)
plt.figure(figsize=(10, 6))
sns.histplot(df_employed_clean['YEARS_EMPLOYED'], bins=50, kde=True)
plt.title('Distribuição dos Anos de Emprego (Excluindo Anomalias)')
plt.xlabel('Anos de Emprego')
plt.ylabel('Contagem de Clientes')

# Salvar o gráfico na pasta 'imagens'
plt.savefig(os.path.join(imagens_dir, 'distribuição_dos_anos_emprego.png'))
plt.close() # Fechar o plot para liberar memória

print("Gráfico 'distribuição_dos_anos_emprego.png' gerado com sucesso!")

Gráfico 'distribuição_dos_anos_emprego.png' gerado com sucesso!


In [60]:
large_values = df_application_train[df_application_train['DAYS_EMPLOYED'] > 10000]['DAYS_EMPLOYED'].unique()
print(np.sort(large_values))

# Verificando a distribuição de TARGET para os valores "normais" de DAYS_EMPLOYED
# Podemos fazer um boxplot ou violinplot para ver a relação com o TARGET
plt.figure(figsize=(10, 6))
sns.violinplot(x='TARGET', y='YEARS_EMPLOYED', data=df_employed_clean)
plt.title('Distribuição dos Anos de Emprego vs. TARGET')
plt.xlabel('Status do Empréstimo (0: Pago, 1: Inadimplente)')
plt.ylabel('Anos de Emprego')


# Salvar o gráfico na pasta 'imagens'
plt.savefig(os.path.join(imagens_dir, 'boxplot_dos_anos_emprego.png'))
plt.close() # Fechar o plot para liberar memória

print("Gráfico 'boxplot_dos_anos_emprego.png' gerado com sucesso!")

[365243]
Gráfico 'boxplot_dos_anos_emprego.png' gerado com sucesso!


In [61]:
# Criar a nova feature binária
df_application_train['DAYS_EMPLOYED_ANOM'] = df_application_train['DAYS_EMPLOYED'] == 365243

# Substituir o valor anômalo por NaN na coluna original DAYS_EMPLOYED
df_application_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace=True)

# Opcional: Converter DAYS_EMPLOYED para anos (e positivo) para melhor interpretabilidade
# Embora não seja necessário para o modelo, é bom para EDA e entendimento humano
df_application_train['YEARS_EMPLOYED'] = df_application_train['DAYS_EMPLOYED'] / -365

# Verificar as primeiras linhas e a nova feature
print("Primeiras linhas de DAYS_EMPLOYED e a nova feature:")
print(df_application_train[['DAYS_EMPLOYED', 'DAYS_EMPLOYED_ANOM', 'YEARS_EMPLOYED']].head())

# Verificar a contagem de anomalias na nova feature
print("\nContagem de valores na nova feature 'DAYS_EMPLOYED_ANOM':")
print(df_application_train['DAYS_EMPLOYED_ANOM'].value_counts())

# Verificar se não há mais o valor 365243 na coluna DAYS_EMPLOYED
print(f"\nContagem do valor 365243 em 'DAYS_EMPLOYED' após tratamento: {(df_application_train['DAYS_EMPLOYED'] == 365243).sum()}")

Primeiras linhas de DAYS_EMPLOYED e a nova feature:
   DAYS_EMPLOYED  DAYS_EMPLOYED_ANOM  YEARS_EMPLOYED
0         -637.0               False        1.745205
1        -1188.0               False        3.254795
2         -225.0               False        0.616438
3        -3039.0               False        8.326027
4        -3038.0               False        8.323288

Contagem de valores na nova feature 'DAYS_EMPLOYED_ANOM':
DAYS_EMPLOYED_ANOM
False    252137
True      55374
Name: count, dtype: int64

Contagem do valor 365243 em 'DAYS_EMPLOYED' após tratamento: 0


/tmp/ipython-input-61-530986364.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_application_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace=True)


In [62]:
missing_values_current = df_application_train.isnull().sum()
missing_percentage_current = (missing_values_current / len(df_application_train)) * 100
missing_df_current = pd.DataFrame({
    'Missing Count': missing_values_current,
    'Missing Percentage (%)': missing_percentage_current
})
missing_df_current = missing_df_current[missing_df_current['Missing Count'] > 0].sort_values(
    by='Missing Percentage (%)', ascending=False
)

print("Valores Ausentes Atuais por Coluna:")
print(missing_df_current)



Valores Ausentes Atuais por Coluna:
                          Missing Count  Missing Percentage (%)
COMMONAREA_AVG                   214865               69.872297
COMMONAREA_MODE                  214865               69.872297
COMMONAREA_MEDI                  214865               69.872297
NONLIVINGAPARTMENTS_AVG          213514               69.432963
NONLIVINGAPARTMENTS_MEDI         213514               69.432963
...                                 ...                     ...
EXT_SOURCE_2                        660                0.214626
AMT_GOODS_PRICE                     278                0.090403
AMT_ANNUITY                          12                0.003902
CNT_FAM_MEMBERS                       2                0.000650
DAYS_LAST_PHONE_CHANGE                1                0.000325

[69 rows x 2 columns]


In [63]:
# Create a bar plot of missing values percentage
plt.figure(figsize=(12, 8))
sns.barplot(x=missing_df_current.index, y='Missing Percentage (%)', data=missing_df_current)
plt.xticks(rotation=90)
plt.xlabel('Columns')
plt.ylabel('Missing Percentage (%)')
plt.title('Percentage of Missing Values per Column')
plt.tight_layout()

# Save the plot
plt.savefig(os.path.join(imagens_dir, 'missing_values_percentage_bar_plot.png'))
plt.close()

print("Gráfico 'missing_values_percentage_bar_plot.png' gerado com sucesso!")

Gráfico 'missing_values_percentage_bar_plot.png' gerado com sucesso!


In [64]:
df_application_train['OWN_CAR_AGE_IS_NULL'] = df_application_train['OWN_CAR_AGE'].isnull()

# Contar a frequência de FLAG_OWN_CAR para quando OWN_CAR_AGE é ausente
print("\nContagem de 'FLAG_OWN_CAR' quando 'OWN_CAR_AGE' é nulo:")
print(df_application_train[df_application_train['OWN_CAR_AGE_IS_NULL']]['FLAG_OWN_CAR'].value_counts())

# Contar a frequência de FLAG_OWN_CAR para quando OWN_CAR_AGE NÃO é nulo
print("\nContagem de 'FLAG_OWN_CAR' quando 'OWN_CAR_AGE' NÃO é nulo:")
print(df_application_train[~df_application_train['OWN_CAR_AGE_IS_NULL']]['FLAG_OWN_CAR'].value_counts())


Contagem de 'FLAG_OWN_CAR' quando 'OWN_CAR_AGE' é nulo:
FLAG_OWN_CAR
N    202924
Y         5
Name: count, dtype: int64

Contagem de 'FLAG_OWN_CAR' quando 'OWN_CAR_AGE' NÃO é nulo:
FLAG_OWN_CAR
Y    104582
Name: count, dtype: int64


In [65]:
def analyze_missing_vs_flag(df, missing_col, flag_cols):
    """
    Analisa a relação entre a presença de valores ausentes em uma coluna
    e a distribuição de valores em colunas 'flag' (binárias ou categóricas).

    Args:
        df (pd.DataFrame): O DataFrame a ser analisado.
        missing_col (str): O nome da coluna a ser investigada por valores ausentes.
        flag_cols (list): Uma lista de nomes de colunas 'flag' para correlacionar.
    """
    print(f"--- Análise para a coluna: '{missing_col}' ---")

    # Contagem e porcentagem de valores ausentes na coluna principal
    missing_count = df[missing_col].isnull().sum()
    total_rows = len(df)
    missing_percentage = (missing_count / total_rows) * 100

    print(f"Valores ausentes em '{missing_col}': {missing_count}")
    print(f"Porcentagem de ausentes em '{missing_col}': {missing_percentage:.2f}%\n")

    # Criar uma coluna auxiliar para identificar os nulos
    df_temp = df.copy() # Criar uma cópia para não modificar o DataFrame original
    df_temp[f'{missing_col}_IS_NULL'] = df_temp[missing_col].isnull()

    for flag_col in flag_cols:
        print(f"--- Relação com '{flag_col}' ---")
        if flag_col not in df.columns:
            print(f"A coluna '{flag_col}' não foi encontrada no DataFrame. Pulando.")
            continue

        # Contagem de FLAG_COL para quando MISSING_COL é nulo
        print(f"\nContagem de '{flag_col}' quando '{missing_col}' é nulo:")
        print(df_temp[df_temp[f'{missing_col}_IS_NULL']][flag_col].value_counts(dropna=False)) # dropna=False para incluir NaNs

        # Contagem de FLAG_COL para quando MISSING_COL NÃO é nulo
        print(f"\nContagem de '{flag_col}' quando '{missing_col}' NÃO é nulo:")
        print(df_temp[~df_temp[f'{missing_col}_IS_NULL']][flag_col].value_counts(dropna=False))
        print("-" * 30 + "\n")



In [66]:
# --- Exemplo de uso com o seu DataFrame ---

# Colunas com muitos missing values que gostaríamos de investigar
cols_to_investigate = [
    'OWN_CAR_AGE',
    'COMMONAREA_AVG',
    'FLOORSMIN_AVG'
]

# Colunas binárias ou categóricas que podem explicar os missing values
flag_features = [
    'FLAG_OWN_CAR',
    'FLAG_OWN_REALTY'
]

# Executar a função para cada coluna com missing values que queremos investigar
for col in cols_to_investigate:
    analyze_missing_vs_flag(df_application_train, col, flag_features)

--- Análise para a coluna: 'OWN_CAR_AGE' ---
Valores ausentes em 'OWN_CAR_AGE': 202929
Porcentagem de ausentes em 'OWN_CAR_AGE': 65.99%

--- Relação com 'FLAG_OWN_CAR' ---

Contagem de 'FLAG_OWN_CAR' quando 'OWN_CAR_AGE' é nulo:
FLAG_OWN_CAR
N    202924
Y         5
Name: count, dtype: int64

Contagem de 'FLAG_OWN_CAR' quando 'OWN_CAR_AGE' NÃO é nulo:
FLAG_OWN_CAR
Y    104582
Name: count, dtype: int64
------------------------------

--- Relação com 'FLAG_OWN_REALTY' ---

Contagem de 'FLAG_OWN_REALTY' quando 'OWN_CAR_AGE' é nulo:
FLAG_OWN_REALTY
Y    140954
N     61975
Name: count, dtype: int64

Contagem de 'FLAG_OWN_REALTY' quando 'OWN_CAR_AGE' NÃO é nulo:
FLAG_OWN_REALTY
Y    72358
N    32224
Name: count, dtype: int64
------------------------------

--- Análise para a coluna: 'COMMONAREA_AVG' ---
Valores ausentes em 'COMMONAREA_AVG': 214865
Porcentagem de ausentes em 'COMMONAREA_AVG': 69.87%

--- Relação com 'FLAG_OWN_CAR' ---

Contagem de 'FLAG_OWN_CAR' quando 'COMMONAREA_AVG' é nulo:

In [67]:
# Contar os NaN antes do tratamento
print(f"Número de NaN em 'OWN_CAR_AGE' antes do tratamento: {df_application_train['OWN_CAR_AGE'].isnull().sum()}")

# Preencher os valores ausentes de OWN_CAR_AGE com 0
df_application_train['OWN_CAR_AGE'].fillna(0, inplace=True)

# Contar os NaN depois do tratamento
print(f"Número de NaN em 'OWN_CAR_AGE' depois do tratamento: {df_application_train['OWN_CAR_AGE'].isnull().sum()}")

# Verificar algumas linhas para ver o efeito (pegando exemplos que antes eram NaN)
# Para isso, seria ideal ter salvo os índices originais com NaN, mas podemos inspecionar
print("\nVerificando o efeito do preenchimento em 'OWN_CAR_AGE' (amostra de clientes que não possuem carro):")
# Pegando clientes que tinham FLAG_OWN_CAR = 'N' e agora deveriam ter OWN_CAR_AGE = 0
print(df_application_train[df_application_train['FLAG_OWN_CAR'] == 'N'][['FLAG_OWN_CAR', 'OWN_CAR_AGE']].sample(5, random_state=42))

Número de NaN em 'OWN_CAR_AGE' antes do tratamento: 202929
Número de NaN em 'OWN_CAR_AGE' depois do tratamento: 0

Verificando o efeito do preenchimento em 'OWN_CAR_AGE' (amostra de clientes que não possuem carro):
       FLAG_OWN_CAR  OWN_CAR_AGE
188565            N          0.0
4623              N          0.0
8360              N          0.0
278378            N          0.0
66519             N          0.0


/tmp/ipython-input-67-1925813925.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_application_train['OWN_CAR_AGE'].fillna(0, inplace=True)


In [68]:
# Separar colunas numéricas e categóricas que ainda possuem NA
numeric_cols_with_na = df_application_train[missing_df_current.index].select_dtypes(include=np.number).columns.tolist()
categorical_cols_with_na = df_application_train[missing_df_current.index].select_dtypes(include='object').columns.tolist()

print("\nColunas Numéricas com NA (para MICE ou Mediana):")
print(numeric_cols_with_na)

print("\nColunas Categóricas com NA (para imputar 'Missing' e One-Hot Encoding):")
print(categorical_cols_with_na)

# --- Implementação - Imputação Simples para Categóricas com NA ---
print("\n--- Imputando 'Missing' em Colunas Categóricas com NA ---")
for col in categorical_cols_with_na:
    df_application_train[col].fillna('Missing', inplace=True)
    print(f"Coluna '{col}' preenchida. NaN restantes: {df_application_train[col].isnull().sum()}")

# Verificar se não há mais NaN em colunas categóricas
print("\nVerificando NaN em colunas categóricas após imputação:")
print(df_application_train[categorical_cols_with_na].isnull().sum().sum())


Colunas Numéricas com NA (para MICE ou Mediana):
['COMMONAREA_AVG', 'COMMONAREA_MODE', 'COMMONAREA_MEDI', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_MODE', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAPARTMENTS_MODE', 'LIVINGAPARTMENTS_AVG', 'FLOORSMIN_MODE', 'FLOORSMIN_AVG', 'FLOORSMIN_MEDI', 'YEARS_BUILD_MEDI', 'YEARS_BUILD_MODE', 'YEARS_BUILD_AVG', 'OWN_CAR_AGE', 'LANDAREA_AVG', 'LANDAREA_MEDI', 'LANDAREA_MODE', 'BASEMENTAREA_MEDI', 'BASEMENTAREA_MODE', 'BASEMENTAREA_AVG', 'EXT_SOURCE_1', 'NONLIVINGAREA_MODE', 'NONLIVINGAREA_MEDI', 'NONLIVINGAREA_AVG', 'ELEVATORS_AVG', 'ELEVATORS_MODE', 'ELEVATORS_MEDI', 'APARTMENTS_MODE', 'APARTMENTS_AVG', 'APARTMENTS_MEDI', 'ENTRANCES_MEDI', 'ENTRANCES_MODE', 'ENTRANCES_AVG', 'LIVINGAREA_AVG', 'LIVINGAREA_MODE', 'LIVINGAREA_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMAX_MODE', 'FLOORSMAX_AVG', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BEGINEXPLUATATION_AVG', 'TOTALAREA_MODE', 'EXT_SOURCE_3', 'YEARS_EMPLOYED'

/tmp/ipython-input-68-3101004666.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_application_train[col].fillna('Missing', inplace=True)


0


In [69]:
# Calcular o número de valores ausentes por coluna
missing_values = df_application_train.isnull().sum()
# Calcular a porcentagem de valores ausentes
missing_percentage = (missing_values / len(df_application_train)) * 100

# Criar um DataFrame para visualizar os resultados
missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing Percentage (%)': missing_percentage
})

In [70]:
# Filtrar apenas as colunas com valores ausentes e ordenar
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values(
    by='Missing Percentage (%)', ascending=False)

# Exibir as primeiras linhas do DataFrame de valores ausentes
print("Valores Ausentes por Coluna:")
print(missing_df.head(20)) # Mostra as 20 colunas com mais valores ausentes

Valores Ausentes por Coluna:
                          Missing Count  Missing Percentage (%)
COMMONAREA_AVG                   214865               69.872297
COMMONAREA_MODE                  214865               69.872297
COMMONAREA_MEDI                  214865               69.872297
NONLIVINGAPARTMENTS_AVG          213514               69.432963
NONLIVINGAPARTMENTS_MODE         213514               69.432963
NONLIVINGAPARTMENTS_MEDI         213514               69.432963
LIVINGAPARTMENTS_MEDI            210199               68.354953
LIVINGAPARTMENTS_AVG             210199               68.354953
LIVINGAPARTMENTS_MODE            210199               68.354953
FLOORSMIN_MEDI                   208642               67.848630
FLOORSMIN_AVG                    208642               67.848630
FLOORSMIN_MODE                   208642               67.848630
YEARS_BUILD_MODE                 204488               66.497784
YEARS_BUILD_AVG                  204488               66.497784
YEARS_BUILD

In [71]:
# Identificar colunas categóricas (object dtype)
categorical_cols = df_application_train.select_dtypes(include='object').columns.tolist()

# Aplicar One-Hot Encoding nas colunas categóricas
df_application_train = pd.get_dummies(df_application_train, columns=categorical_cols, dummy_na=False) # dummy_na=False pois já tratamos NaNs

print(f"Shape do DataFrame após One-Hot Encoding: {df_application_train.shape}")

# Verificar se ainda existem colunas 'object'
print("\nVerificando se ainda existem colunas do tipo 'object':")
print(df_application_train.select_dtypes(include='object').columns)

# Recalcular e exibir os missing values restantes (apenas numéricos agora)
missing_values_final_check = df_application_train.isnull().sum()
missing_percentage_final_check = (missing_values_final_check / len(df_application_train)) * 100
missing_df_final_check = pd.DataFrame({
    'Missing Count': missing_values_final_check,
    'Missing Percentage (%)': missing_percentage_final_check
})
missing_df_final_check = missing_df_final_check[missing_df_final_check['Missing Count'] > 0].sort_values(
    by='Missing Percentage (%)', ascending=False
)

print("\nValores Ausentes Restantes por Coluna (após One-Hot Encoding):")
print(missing_df_final_check)

Shape do DataFrame após One-Hot Encoding: (307511, 255)

Verificando se ainda existem colunas do tipo 'object':
Index([], dtype='object')

Valores Ausentes Restantes por Coluna (após One-Hot Encoding):
                          Missing Count  Missing Percentage (%)
COMMONAREA_AVG                   214865               69.872297
COMMONAREA_MODE                  214865               69.872297
COMMONAREA_MEDI                  214865               69.872297
NONLIVINGAPARTMENTS_AVG          213514               69.432963
NONLIVINGAPARTMENTS_MODE         213514               69.432963
...                                 ...                     ...
EXT_SOURCE_2                        660                0.214626
AMT_GOODS_PRICE                     278                0.090403
AMT_ANNUITY                          12                0.003902
CNT_FAM_MEMBERS                       2                0.000650
DAYS_LAST_PHONE_CHANGE                1                0.000325

[62 rows x 2 columns]


In [72]:
# Identificar colunas numéricas com NaN restantes
numeric_cols_with_na = df_application_train.select_dtypes(include=np.number).columns[df_application_train.select_dtypes(include=np.number).isnull().any()].tolist()

# Classificar as colunas para imputação
cols_for_mice = []
cols_for_median = []

# Usaremos um limite arbitrário de 30% para decidir entre mediana e MICE
# As colunas de imóvel que tinham ~70% de NA irão para MICE
# Outras colunas como AMT_ANNUITY, AMT_GOODS_PRICE, etc., têm menos NA
# (Você pode ajustar esse limite ou inspecionar manualmente as porcentagens para decidir)

# Recalcular porcentagens de NA para as colunas numéricas restantes
current_missing_percentage = (df_application_train[numeric_cols_with_na].isnull().sum() / len(df_application_train)) * 100

for col in numeric_cols_with_na:
    if current_missing_percentage[col] > 30: # Ex: para as colunas de imóvel (70% NA)
        cols_for_mice.append(col)
    else: # Para outras colunas com menor % de NA
        cols_for_median.append(col)

print(f"\nColunas para imputação com MEDIANA: {cols_for_median}")
print(f"Colunas para imputação com MICE: {cols_for_mice}")

# --- Implementação - Imputação por Mediana ---
print("\n--- Imputando colunas com Mediana ---")
for col in cols_for_median:
    median_val = df_application_train[col].median()
    df_application_train[col].fillna(median_val, inplace=True)
    print(f"Coluna '{col}' preenchida com mediana ({median_val:.2f}). NaN restantes: {df_application_train[col].isnull().sum()}")




Colunas para imputação com MEDIANA: ['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_EMPLOYED', 'CNT_FAM_MEMBERS', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'YEARS_EMPLOYED']
Colunas para imputação com MICE: ['EXT_SOURCE_1', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE

/tmp/ipython-input-72-2671286041.py:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_application_train[col].fillna(median_val, inplace=True)


In [73]:
 #--- Implementação - Imputação com MICE ---
print("\n--- Imputando colunas com MICE (IterativeImputer) ---")
if cols_for_mice: # Só executa se houver colunas para MICE
    # A IterativeImputer funciona melhor em uma cópia do DataFrame ou em um subconjunto
    # A imputação é feita in-place no array numpy, então usaremos a cópia ou selecionaremos apenas as colunas
    imputer = IterativeImputer(max_iter=10, random_state=0) # max_iter é o número de iterações

    # Aplicar MICE apenas nas colunas designadas
    # Convertemos para numpy array para o imputer, e depois de volta para DataFrame
    # Cuidado: Isso vai imputar em TODO o DataFrame ou em um subconjunto que você passar.
    # É mais seguro imputar apenas nas colunas específicas e depois atribuir de volta.

    # Criar um sub-DataFrame apenas com as colunas para MICE
    df_mice_subset = df_application_train[cols_for_mice]

    # Aplicar o imputer
    imputed_data = imputer.fit_transform(df_mice_subset)

    # Atribuir os valores imputados de volta ao DataFrame original
    df_application_train[cols_for_mice] = imputed_data

    print(f"Colunas {cols_for_mice} imputadas usando MICE.")
else:
    print("Nenhuma coluna identificada para imputação MICE.")


# --- Verificação Final ---
print("\n--- Verificação Final de Valores Ausentes ---")
final_missing_check = df_application_train.isnull().sum()
final_missing_check = final_missing_check[final_missing_check > 0]
if final_missing_check.empty:
    print("Todos os valores ausentes foram tratados! 🎉")
else:
    print("Ainda existem valores ausentes:")
    print(final_missing_check)

print(f"\nShape final do DataFrame após todos os tratamentos de valores ausentes: {df_application_train.shape}")


--- Imputando colunas com MICE (IterativeImputer) ---


/usr/local/lib/python3.11/dist-packages/sklearn/impute/_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Colunas ['EXT_SOURCE_1', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 'TOTALAREA_MODE'] imputadas usando MICE.

--- Verificação Final de Valores Ausentes ---
Todos os valores au